In [1]:
import DualwormFunctions as dw
import Observables as obs

%load_ext line_profiler
%load_ext memory_profiler

import RunBasis
import argparse
import StartStates as strst
import numpy as np

import yep

In [2]:
nst = 10;
nsm = 10;
nips = 1;
nrps = 1;
nb = 2;
measperiod = 1;
nt = 10
nh = 61
ncores = 1
stroutput = "Profiling_nh{0}_nt{1}_nst{2}_nsm{3}_nips{4}_nrps{5}_ncores{6}_v7_nolprun".format(nh, nt, nst, nsm, nips, nrps, ncores)


In [3]:
def genArgs():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-f', type = str)

    parser.add_argument('--L', type = int, default = 12, help = 'Lattice side size')

    # COUPLINGS
    parser.add_argument('--J1', type = float, default = 1,
                        help = 'NN coupling') # nearest-neighbour coupling
    parser.add_argument('--J2', type = float, default = 0,
                        help = '2nd NN coupling') # 2nd NN coupling
    parser.add_argument('--J3', type = float, default = 0,
                        help = '3rd NN coupling') # 3rd NN coupling
    parser.add_argument('--J4', type = float, default = 0.0,
                        help = '4th NN coupling')
    parser.add_argument('--h', type = float, default = 1.0,
                        help = 'Magnetic field')    
    #NUMBER OF STEPS AND ITERATIONS
    parser.add_argument('--nst', type = int, default = nst,
                        help = 'number of thermalisation steps') # number of thermalisation steps
    parser.add_argument('--nsm', type = int, default = nsm,
                        help = 'number of measurements steps') # number of measurement steps
    parser.add_argument('--nips', type = int, default = nips,
                        help = 'number of worm constructions per MC step')
    parser.add_argument('--nrps', type = int, default = nrps,
                        help = 'number of replica loops per MC step')
    parser.add_argument('--measperiod', type = int, default = measperiod,
                        help = 'number of nips worm building + swaps between measurements')
    parser.add_argument('--nb', type = int, default = nb,
                        help = 'number of bins')

    #PARALLELISATION
    parser.add_argument('--ncores', type = int, default = ncores,
                        help = 'number of threads to use')

    
    # INITIALISATION PARAMETERS
    parser.add_argument('--randominit', default = True, action ='store_true',
                        help = 'intialise the states randomly')
    parser.add_argument('--same', default = False, action = 'store_true',
                        help = '''initialise all temperatures with the same
                        state (debug purposes)''')
    parser.add_argument('--magninit', default = False, action = 'store_true',
                        help = '''initialise all the temperature with
                        one of the m=1/3 GS''')
    parser.add_argument('--magnstripes', default = False, action = 'store_true',
                       help = '''initialise all the temperature with
                       m=1/3 stripes''')
    parser.add_argument('--maxflip', default = False, action = 'store_true',
                       help = '''initialise all the temperature with
                       maximally flippable plateau''')
    parser.add_argument('--loadfromfile', default = False, action = 'store_true',
                       help = '''initialise all the states with
                       results from a previously performed simulations''')
    parser.add_argument('--filename', type = str, default = 'LongerTests_folder',
                       help = '''initialise all the states with
                       results from a previously performed simulations''')
    
    #WORM PARAMETERS
    parser.add_argument('--nmaxiter', type = int, default = 10,
                        help = '''maximal number of segments in a loop update over the
                        size of the lattice (1 = 1times the number of dualbonds in the
                        lattice)''')
    parser.add_argument('--measupdate', default = False, action = 'store_true',
                       help = '''activate to mimic the action of the measuring tip''')
    parser.add_argument('--p', type = float, default = 0.0, 
                       help = '''prob of the measuring tip flipping the spin (number between 0 and 1)''')
    parser.add_argument('--ssf', default = True, action = 'store_true',
                        help = 'activate for single spin flip update')
    parser.add_argument('--alternate', default = True, action = 'store_true',
                        help = 'activate for single spin flip update and dw update')
    parser.add_argument('--checkgs', default =False, action = 'store_true',
                        help = 'activate to debug ssf')
    
    #TEMPERATURE PARAMETERS
    parser.add_argument('--t_list', nargs = '+', type = float, default = [0.05,nt+0.05],
                        help = 'list of limiting temperature values')
    parser.add_argument('--nt_list', nargs = '+', type = int, default = [nt],
                        help = 'list of number of temperatures in between the given limiting temperatures')
    parser.add_argument('--log_tlist', default = True, action='store_true',
                        help = 'state whether you want the temperature be spaced log-like or linear-like (activate if you want log)')
    parser.add_argument('--stat_temps_lims', nargs = '+', type = float,
                        help = '''limiting temperatures for the various ranges of
                        measurements''') 
                        #default will be set to none, and then we can decide what to do later on.
    #MAGNETIC FIELD PARAMETERS
    parser.add_argument('--h_list', nargs = '+', default = [0,6.0],type = float,
                        help = 'list of limiting magnetic field values')
    parser.add_argument('--nh_list', nargs = '+', default = [nh], type = int,
                        help = 'list of number of magnetic fields in between the given limiting temperatures')
    parser.add_argument('--stat_hfields_lims', nargs = '+', type = float,
                help = '''limiting magnetic fields for the various ranges of
                measurements''') 
    #CORRELATIONS PARAMETER
    parser.add_argument('--energy', default = True, action = 'store_true',
                        help = 'activate if you want to save the energy')
    parser.add_argument('--magnetisation', default =True, action = 'store_true',
                        help = 'activate if you want to save the magnetisation')
    parser.add_argument('--charges', default = False, action = 'store_true',
                        help = 'activate if you want to save the charges')
    parser.add_argument('--correlations', default = True, action = 'store_true',
                        help = 'activate if you want to save either central or all correlations')
    parser.add_argument('--all_correlations', default = False, action = 'store_true',
                        help = '''activate if you want to save the correlations for all non-equivalent
                        pairs of sites. Otherwise, will save central correlations.''')
    parser.add_argument('--firstcorrelations', default = False, action = 'store_true',
                        help = 'activate if you want to save first correlations, otherwise will save central')
    #SAVE
    parser.add_argument('--output', type = str, default = stroutput, help = 'saving filename (.pkl will be added)')
    args = parser.parse_args()
    return args

In [4]:
def callRunBasis():
    args = genArgs()
    return RunBasis.main(args)

In [5]:
#%%prun -s  -T prun0
yep.start(stroutput+'.log')
#%lprun -f dw.mcs_swaps
[meanstat, swapst, swapsh, failedupdatesth, failedupdates] = callRunBasis()
yep.stop()

-------------------Initialisation--------------------
./Profiling_nh61_nt10_nst10_nsm10_nips1_nrps1_ncores1_v7_nolprun_folder/backup.hkl
Lattice side size:  12
J1  1
J2  0
J3  0
J3st  0
Couplings extracted
Hamiltonian expression (without field) computed
single spin flip update
alternating ssf and dw update
Number of temperatures:  10
Temperatures: [ 0.05        0.09013235  0.1624768   0.2928883   0.52797419  0.95175106
  1.71567113  3.09274929  5.57513501 10.05      ]
Number of magnetic fields:  61
Magnetic fields:  [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7
 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7 2.8 2.9 3.  3.1 3.2 3.3 3.4 3.5
 3.6 3.7 3.8 3.9 4.  4.1 4.2 4.3 4.4 4.5 4.6 4.7 4.8 4.9 5.  5.1 5.2 5.3
 5.4 5.5 5.6 5.7 5.8 5.9 6. ]
Fully random initialisation =  True
Identical initialisation =  False
Magnetisation initialisation =  False
List of measurements to be performed: ['Energy', 'Magnetisation', 'Si', 'Central_Correlations']
List of field indices to 

In [6]:
print(failedupdatesth/(nst*nips))

[[0.5533179  0.96990741 0.98263889 0.98611111 0.96975309 0.98425926
  0.9882716  0.9871142  0.97438272 0.97376543 0.98796296 0.97044753
  0.98796296 0.98464506 0.98526235 0.98966049 0.98811728 0.98657407
  0.98425926 0.96566358 0.82384259 0.95925926 0.96010802 0.96381173
  0.96381173 0.97052469 0.9695216  0.96890432 0.96820988 0.97145062
  0.97067901 0.96450617 0.97075617 0.96103395 0.96257716 0.96141975
  0.9621142  0.97091049 0.96929012 0.95192901 0.60185185 0.92584877
  0.94984568 0.95231481 0.95223765 0.95084877 0.95185185 0.95216049
  0.95239198 0.95146605 0.95084877 0.95308642 0.95138889 0.95200617
  0.95177469 0.95216049 0.95200617 0.95246914 0.95208333 0.95231481
  0.95185185]
 [0.55516975 0.88078704 0.96118827 0.96512346 0.96929012 0.9691358
  0.9878858  0.98942901 0.98317901 0.9841821  0.96990741 0.97021605
  0.96882716 0.96597222 0.98364198 0.96983025 0.98888889 0.9871142
  0.97785494 0.95501543 0.81473765 0.92962963 0.96111111 0.96728395
  0.96782407 0.96072531 0.96859568 0

In [7]:
print(failedupdates/(nsm*measperiod*nips))

[[0.55185185 0.98341049 1.         1.00069444 1.0007716  1.0007716
  1.0007716  1.0007716  1.0007716  1.0007716  1.0007716  1.0007716
  1.0007716  1.0007716  1.0007716  1.0007716  1.00069444 1.0007716
  1.00069444 0.9970679  0.92608025 0.9962963  1.00046296 1.00061728
  1.0007716  1.0007716  1.0007716  1.0007716  1.0007716  1.0007716
  1.0007716  1.0007716  1.0007716  1.0007716  1.00061728 1.0007716
  1.0007716  1.00069444 0.99976852 0.9912037  0.60308642 0.96790123
  0.99969136 1.0007716  1.0007716  1.0007716  1.0007716  1.0007716
  1.0007716  1.0007716  1.0007716  1.0007716  1.0007716  1.0007716
  1.0007716  1.0007716  1.0007716  1.0007716  1.0007716  1.0007716
  1.0007716 ]
 [0.55524691 0.90532407 0.98904321 0.99945988 1.0003858  1.00061728
  1.0007716  1.0007716  1.0007716  1.0007716  1.0007716  1.0007716
  1.0007716  1.0007716  1.0007716  1.0007716  1.0007716  1.0003858
  0.99768519 0.98518519 0.92947531 0.9808642  0.99660494 1.00007716
  1.00061728 1.0007716  1.00061728 1.0007716

In [8]:
print(4*swapst/(nrps*nsm*measperiod*nh))

[0.67540984 0.5442623  0.4852459  0.28196721 0.         0.
 0.         0.         0.         0.        ]


In [9]:
print(4*swapsh/(nrps*nsm*measperiod*nt))

[0.32 0.16 0.4  0.24 0.64 0.52 0.6  0.56 0.8  0.6  0.68 0.56 0.52 0.44
 0.6  0.36 0.44 0.36 0.36 0.24 0.44 0.2  0.4  0.32 0.44 0.4  0.4  0.48
 0.64 0.56 0.68 0.64 0.68 0.52 0.52 0.56 0.72 0.36 0.4  0.28 0.28 0.12
 0.44 0.32 0.6  0.36 0.6  0.4  0.76 0.48 0.68 0.68 0.96 0.48 0.72 0.6
 1.04 0.64 0.92 0.68 0.28]
